In [21]:
from tensor import TensorSpec, Tensor, TensorShape
from utils.index import Index
from collections.dict import Dict, KeyElement
from random import rand
alias type = DType.float32

In [22]:
@value
struct IntKey(KeyElement):
    var n: Int

    fn __init__(inout self, owned n: Int):
        self.n = n

    fn __hash__(self) -> Int:
        return hash(self.n)

    fn __eq__(self, other: Self) -> Bool:
        return self.n == other.n


let fashion_mnist_key = Dict[IntKey, String]()
fashion_mnist_key[0] = "T-shirt/top"
fashion_mnist_key[1] = "Trouser"
fashion_mnist_key[2] = "Pullover"
fashion_mnist_key[3] = "Dress"
fashion_mnist_key[4] = "Coat"
fashion_mnist_key[5] = "Sandal"
fashion_mnist_key[6] = "Shirt"
fashion_mnist_key[7] = "Sneaker"
fashion_mnist_key[8] = "Bag"
fashion_mnist_key[9] = "Ankle boot"


In [23]:
fn uint8_to_int32(uint8_tensor: Tensor[DType.uint8]) raises -> SIMD[DType.int32, 1]:
    let result = (
        (uint8_tensor[0].to_int() << 24)
        | (uint8_tensor[1].to_int() << 16)
        | (uint8_tensor[2].to_int() << 8)
        | uint8_tensor[3].to_int()
    )
    return SIMD[DType.int32, 1](result)


In [24]:
fn get_slice[type: DType](tensor: Tensor[type], start_index: Int, end_index: Int) raises -> Tensor[type]:
    if end_index < start_index:
        raise "End index more than start index."
    elif end_index == start_index:
        var output_tensor = Tensor[type](1)
        output_tensor[0] = tensor[start_index]

        return output_tensor
    else:
        var output_tensor = Tensor[type](end_index - start_index)
        for i in range(start_index, end_index):
            output_tensor[i - start_index] = tensor[i]
        return output_tensor


In [25]:
fn read_data_as_images(images_path: Path) raises -> Tensor[DType.uint8]:
    if images_path.exists():
        let image_file = images_path.read_bytes().astype[DType.uint8]()
        let num_images = uint8_to_int32(get_slice[DType.uint8](image_file, 4, 8))
        let width = uint8_to_int32(get_slice[DType.uint8](image_file, 8, 12))
        let height = uint8_to_int32(get_slice[DType.uint8](image_file, 12, 16))

        var image_data = get_slice(image_file, 16, image_file.num_elements())

        let images_shape = TensorShape(
            num_images.to_int(), height.to_int(), width.to_int()
        )

        let images = image_data.reshape(images_shape)
        return images
    raise "The images directory does not exist."


In [26]:
fn read_data_as_labels(labels_path: Path) raises -> Tensor[DType.uint8]:
    if labels_path.exists():
        let label_file = labels_path.read_bytes().astype[DType.uint8]()
        let num_labels = uint8_to_int32(get_slice[DType.uint8](label_file, 4, 8))

        let labels = get_slice(label_file, 8, num_labels.to_int() + 8)

        return labels
    raise "The labels directory does not exist."


In [27]:
fn tensor_print[type: DType](index: Int, tensor: Tensor[type]):
    var cur_line: String

    fn get_char_for_pixel(pixel_value: Int) -> String:
        if pixel_value == 0:
            return " "
        elif pixel_value < 32:
            return "."
        elif pixel_value < 64:
            return ","
        elif pixel_value < 96:
            return ":"
        elif pixel_value < 128:
            return ";"
        elif pixel_value < 160:
            return "o"
        elif pixel_value < 192:
            return "O"
        elif pixel_value < 224:
            return "X"
        else:
            return "#"

    for j in range(tensor.shape()[1]):
        cur_line = ""
        for k in range(tensor.shape()[2]):
            cur_line += get_char_for_pixel(tensor[Index(index, j, k)].to_int()) + " "
        print(cur_line)


In [28]:
let base_dir = "/Users/tprazak/Documents/seminary_work_nn/MNIST_in_mojo/"
let image_path_mnist = base_dir + "mnist/train-images.idx3-ubyte"
let labels_path_mnist = base_dir + "mnist/train-labels.idx1-ubyte"
let image_path_fashion = base_dir + "fashion_mnist/train-images-idx3-ubyte"
let labels_path_fashion = base_dir + "fashion_mnist/train-labels-idx1-ubyte"
let images = read_data_as_images(image_path_mnist)
let labels = read_data_as_labels(labels_path_mnist)
let fashion_images = read_data_as_images(image_path_fashion)
let fashion_labels = read_data_as_labels(labels_path_fashion)


In [29]:
fn matmul(first_matrix: Tensor[type], second_matrix: Tensor[type]) raises -> Tensor[type]:
    let f_m = first_matrix.astype[type]()
    let s_m = second_matrix.astype[type]()
    if first_matrix.rank() != 2 or second_matrix.rank() != 2:
        raise 'At least one of the tensors is not a matrix'
    if first_matrix.dim(1) != second_matrix.dim(0):
        raise 'Then matrices are not compatible for matrix multiplication'

    let o_m_rows = f_m.dim(0)
    let o_m_columns = s_m.dim(1)
    var o_m = Tensor[type](o_m_rows * o_m_columns)
    for i in range(o_m_rows):
        for j in range(f_m.dim(1)):
            for k in range(o_m_columns):
                o_m[i*o_m_columns+k] += f_m[i, j] * s_m[j, k]
    return o_m.reshape(TensorShape(o_m_rows, o_m_columns))

In [30]:
alias nelts = simdwidthof[type]()
fn matmul_vectorized(first_matrix: Tensor[type], second_matrix: Tensor[type]) raises -> Tensor[type]:
    let f_m = first_matrix.astype[type]()
    let s_m = second_matrix.astype[type]()
    if first_matrix.rank() != 2 or second_matrix.rank() != 2:
        raise 'At least one of the tensors is not a matrix'
    if first_matrix.dim(1) != second_matrix.dim(0):
        raise 'Then matrices are not compatible for matrix multiplication'

    let o_m_rows = f_m.dim(0)
    let o_m_cols = s_m.dim(1)
    var o_m = Tensor[type](o_m_rows * o_m_cols)
    for i in range(o_m_rows):
        for j in range(f_m.dim(1)):
            for kv in range(0, nelts*(o_m_cols//nelts), nelts):
                o_m.simd_store[nelts](i*o_m_cols+kv, o_m.simd_load[nelts](i*o_m_cols+kv) + f_m.simd_load[1](i*f_m.dim(1)+j) * s_m.simd_load[nelts](j*s_m.dim(1)+kv))
            for k in range(nelts*(o_m_cols//nelts), o_m_cols):
                o_m.simd_store[1](i*o_m_cols+k, o_m.simd_load[1](i*o_m_cols+k) + f_m.simd_load[1](i*f_m.dim(1)+j) * s_m.simd_load[1](j*s_m.dim(1)+k))
    return o_m.reshape(TensorShape(o_m_rows, o_m_cols))

In [31]:
fn add_biases(output: Tensor[type], biases: Tensor[type]) raises -> Tensor[type]:
    let samples = output.dim(0)
    let num_neurons = output.dim(1)

    var biases_extended = Tensor[type](output.shape())
    
    if num_neurons != biases.num_elements():
        raise 'Number of neurons does not match.'
    for i in range(samples):
        for j in range(0, nelts*(num_neurons//nelts), nelts):
            biases_extended.simd_store[nelts](i*num_neurons+j, biases.simd_load[nelts](j))
        for k in range(nelts*(num_neurons//nelts), num_neurons):
            biases_extended.simd_store[1](i*num_neurons+k, biases.simd_load[1](k))
    return output + biases_extended


In [32]:
fn transpose(matrix: Tensor[type]) raises -> Tensor[type]:
    if matrix.rank() != 2:
        raise 'Not a 2D matrix'
    var transposed_m = Tensor[type](TensorShape(matrix.dim(1), matrix.dim(0)))
    for i in range(matrix.dim(0)):
        for j in range(matrix.dim(1)):
            transposed_m.simd_store[1](j*matrix.dim(0)+i, matrix.simd_load[1](i*matrix.dim(1)+j))
    return transposed_m    

In [37]:
fn sum_dvalues_on_cols(dvalues: Tensor[type]) -> Tensor[type]:
    let rows = dvalues.dim(0)
    let cols = dvalues.dim(1)

    var sum_tensor = Tensor[type](cols)
    

    for i in range(rows):
        for j in range(0, nelts*(cols//nelts), nelts):
            sum_tensor.simd_store[nelts](j, dvalues.simd_load[nelts](i*cols+j) + sum_tensor.simd_load[nelts](j))
        for k in range(nelts*(cols//nelts), cols):
            sum_tensor.simd_store[1](k, dvalues.simd_load[1](i*cols+k) + sum_tensor.simd_load[1](k))
    return sum_tensor

In [42]:
struct LayerDense:
    var input: Tensor[type]
    var output: Tensor[type]
    var weights: Tensor[type]
    var biases: Tensor[type]
    var dweights: Tensor[type]
    var dbiases: Tensor[type]
    var dinputs: Tensor[type]

    fn __init__(inout self, num_inputs: Int, num_neurons: Int):
        self.weights = 0.01 * rand[type](num_inputs, num_neurons)
        self.biases = Tensor[type](1, num_neurons)
        self.input = Tensor[type]()
        self.output = Tensor[type]()
        self.dweights = Tensor[type]()
        self.dbiases = Tensor[type]()
        self.dinputs = Tensor[type]()
    
    fn forward(inout self, inputs: Tensor[type]) raises:
        self.output = matmul_vectorized(inputs, self.weights)
        self.output = add_biases(self.output, self.biases)
        self.input = inputs

    
    fn backward(inout self, dvalues: Tensor[type]) raises:
        self.dweights = matmul_vectorized(transpose(self.input), dvalues)
        self.dinputs = matmul_vectorized(dvalues, transpose(self.weights))
        self.dbiases = sum_dvalues_on_cols(dvalues)

In [44]:
fn tensor_max(input: Tensor[type], value: SIMD[type, 1]) -> Tensor[type]:
    var output = Tensor[type](input.shape())
    let cols = input.dim(1)
    for i in range(input.dim(0)):
        for j in range(0, nelts*(cols//nelts), nelts):
            output.simd_store[nelts](i*cols+j, output.simd_load[nelts](i*cols+j).max(value))
        for k in range(nelts*(cols//nelts), cols):
            output.simd_store[1](i*cols+k, output.simd_load[1](i*cols+k).max(value))
    return output

In [ ]:
struct ActivationReLU:
    var input: Tensor[type]
    var output: Tensor[type]
    var dinputs: Tensor[type]

    fn __init__(inout self):
        self.input = Tensor[type]()
        self.output = Tensor[type]()
        self.dinputs = Tensor[type]()

    fn forward(inout self, inputs: Tensor[type]):
        self.output = tensor_max(inputs, 0)
        self.input = inputs

    def backward(inout self, d_values: Tensor[type]):
        pass

In [31]:
var inputs = rand[type](12)
print(inputs)
inputs = inputs.reshape(TensorShape(4, 3))
print(inputs)
var d1 = LayerDense(3, 5)
d1.forward(inputs)
print(d1.output.shape())


Tensor([[0.51701498031616211, 0.73513919115066528, 0.742706298828125, ..., 0.76813429594039917, 0.81904655694961548, 0.43459099531173706]], dtype=float32, shape=12)
Tensor([[0.51701498031616211, 0.73513919115066528, 0.742706298828125],
[0.31978964805603027, 0.17441794276237488, 0.6653144359588623],
[0.25154781341552734, 0.38916084170341492, 0.54644107818603516],
[0.76813429594039917, 0.81904655694961548, 0.43459099531173706]], dtype=float32, shape=4x3)
4x5


In [43]:
let a = SIMD[type, 4](-1, 3, 4, -5)
let b = SIMD[type, 1](0)
print(a.max(b))

[0.0, 3.0, 4.0, 0.0]
